In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from nltk import pos_tag
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
##SCRAPER CODE
# names = []
# reviews = []
# grades = []
# url = 'https://www.temptalia.com/category/reviews/page/1035/'
# response = requests.get(url)
# page = response.text
# soup = BeautifulSoup(page, "lxml")
# product_links = []
# links = soup.find_all(class_='btn btn-secondary btn-lg more-link')
# for link in links:
#     product_links.append(link.get('href'))
# for link in product_links:
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text, "lxml")
#     product_names = soup.find_all(class_ = 'hidden-md-up text-xs-center text-md-left text-uppercase pt-1 mb-2')
#     product_reviews = (soup.find_all('p'))
#     product_grades = (soup.find_all(class_ = 'glossover-grade f-7 bold'))
#     counter = 0
#     for review in product_reviews:
#         if '$' in review.text and len(review.text) > 35:
#             reviews.append(review.text)
#             names.append(product_names[counter].text)
#             grades.append(product_grades[counter].text)
#             counter += 1
# new_df = pd.DataFrame(data = {'names': names, 'reviews': reviews, 'grades': grades})
# new_df.to_csv('temp_sheet.csv')
# new_df = new_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# new_df.tail()

In [3]:
new_df = pd.read_csv('output.csv', encoding = "ISO-8859-1")

In [4]:
new_df.tail()

,Brand,Name,Product,Review,Grade
5168,MAC,OVERSPICED,Lipgloss,"a dirty muted coral. ItÕs a warm-toned, light-...",C+
5169,MAYBELLINE,NUDE ILLUSION (060),Lipgloss,"light-medium peachy beige with soft, warm unde...",B
5170,SURRATT BEAUTY,AU NATURELLE,Lipgloss,a universal neutral with a kiss of gold. ItÕs ...,C+
5171,CHANEL,SENSUEL (11),Lipgloss,a peachy brown with a very fine gold shimmer. ...,B
5172,NARS,BELLE DE JOUR,Lipstick,"a light-medium, yellowed peach with warm under...",D


In [5]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [6]:
clean = new_df['Review'].apply(removeNonAscii)

In [7]:
new_df['Review'] = clean

In [8]:
new_df.replace('-', ' ', regex=True, inplace=True)

In [9]:
new_df.sample(n=20)

,Brand,Name,Product,Review,Grade
3214,ANASTASIA,SANGRIA,Eyeshadow,a medium dark reddish plum with warm undertone...,A
5108,MAC,FASHION FORCE,Lipstick,"a soft tangerine orange. Its a muted, medium ...",B+
4751,BENEFIT,ROCKATEUR,Blush,"a soft, peachy coral with warm undertones and ...",B+
2961,MARC JACOBS BEAUTY,FINE(WINE),Eyeliner,"a deep red with subtle, cool undertones and a ...",B+
4863,NARS,OAXACA,Eyeshadow,"a quartz. Its a muted, medium dark taupe brown...",F
5113,SEPHORA,ROSE WOOD (85),Lipstick,"a medium dark pink with subtle, warm undertone...",A
301,BITE BEAUTY,LIBRA,Lipstick,"a deep, chocolatly brown with warm, yellowish ...",A+
202,MARC JACOBS BEAUTY,BLITZ GLITZ,Eyeshadow,"a medium dark plum with warm, brownish underto...",B+
4720,MARC JACOBS BEAUTY,THE FREE SPIRIT (224),Palette,twenty eyeshadows with two being repeats (thou...,A
846,CHANEL,ULTRA FLESH (838),Eyeshadow,"a soft, pink coral with warm undertones and a ...",B


In [60]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cv_data = count_vectorizer.fit_transform(new_df['Review'])
lsa_ = TruncatedSVD(n_components=1428)
lsa_data = lsa_.fit_transform(cv_data)

In [62]:
new_vec = lsa_.transform(count_vectorizer.transform(['a new, limited edition neutral-hued palette that leans slightly more mauve/plum than overtly warm-toned. Of the three palettes launched in the Art Library collection, this has the most repeated shades (past releases and/or permanent shades). If you feel like a lot of matte eyeshadows are too powdery, you may you enjoy MAC’s formula, which tends to be firmer, a little stiffer and better for gradually building color with little to no fallout. Most of the mattes were blendable, but the darker shades–Sugar Plum and Dance in the Dark–could have been easier to soften along the edges. LES Artiste had some fallout and a tendency to emphasize my lid texture, so I’d recommend using fingertips or a wet brush to apply it, and I felt like Nude Model was overly slippery.']))
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(lsa_data)
results = nn.kneighbors(new_vec, n_neighbors=5)


In [15]:
for i in range(len(results[1][0])):
        print(new_df.iloc[results[1][0][i]])

Brand                                                    MAC
Name                                  THE BEST OF EVERYTHING
Product                                              Palette
Review     a new, limited edition smoky palette. I love t...
Grade                                                     B 
Name: 3112, dtype: object
Brand                                                    MAC
Name                                               EYE Z YOU
Product                                              Palette
Review     a pink and plum hued palette with a subdued gr...
Grade                                                      B
Name: 3808, dtype: object
Brand                                              TOO FACED
Name                                            NATURAL LOVE
Product                                              Palette
Review     30 eyeshadows in varying shades of pinks, gold...
Grade                                                     B 
Name: 2715, dtype: object
Brand  

In [58]:
def get_nearest_prods():
    new_vec = lsa_.transform(count_vectorizer.transform([input()]))
    results = nn.kneighbors(new_vec, n_neighbors=5)
    rec_strings = []
    for i in range(len(results[1][0])):
        rec_strings.append(f"Try {new_df.iloc[results[1][0][i]]['Name']}, a {new_df.iloc[results[1][0][i]]['Product']} by {new_df.iloc[results[1][0][i]]['Brand']}.")
    return rec_strings

    

In [120]:
get_nearest_prods()

KVD Fetish blush/highlighter palette ++  this has been on sale (clearance?) for $21.50 lately, that's what I paid. I've only had a chance to try the lavender blush and h/l. But I found it lovely and long-lasting. Of course, the h/l is glittery. But I have LOADS of h/l and am fine with this one being different. The compact is lovely, embroidered (I suppose it could get dirty, mine don't, I don't know why, I am not a neatnik). More than worth the price for me.


['Try PLUMAGRANATE, a Palette by TOO FACED.',
 "Try SOLEIL D'AMBRE, a Palette by TOM FORD BEAUTY.",
 'Try NOONER, a Palette by URBAN DECAY.',
 'Try SOPHISTICATE, a Palette by MAC.',
 'Try SHATTERED, a Palette by URABN DECAY.']

In [30]:
import pickle
pickle.dump(count_vectorizer, open('vectorizer.pkl', 'wb'))

In [31]:
pickle.dump(lsa_, open('dim_red.pkl', 'wb'))

In [32]:
pickle.dump(nn, open('model.pkl', 'wb'))

In [33]:
pickle.dump(new_df, open('reviews.pkl', 'wb'))